## Meshed up swarms

Using the shadow particle fanciness to achieve great things

In [1]:
import underworld as uw
from underworld import function as fn

import glucifer

uw.matplotlib_inline()

# import matplotlib.pyplot as pyplot
# import matplotlib.pylab as pylab
import numpy as np
import math
import os



/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:

workdir = os.path.abspath(".")
outputPath = os.path.join(workdir,"MeshSwarms/")

if uw.rank() == 0:
    if not os.path.exists(outputPath):
        os.makedirs(outputPath)
    
uw.barrier()   


Create mesh and finite element variables
------

Note: the use of a pressure-sensitive rheology suggests that it is important to use a Q2/dQ1 element 

In [3]:
minX  =  0.0;   minY = 0.0
maxX  =  1.0;   maxY = 1.0


resX = 32
resY = 32

elementType="Q2/dPc1"  # This is enough for a test but not to use the code in anger

mesh = uw.mesh.FeMesh_Cartesian( elementType = (elementType), 
                                 elementRes  = ( resX, resY), 
                                 minCoord    = ( minX, minY), 
                                 maxCoord    = ( maxX, maxY),
                                 periodic    = [False, False]  ) 


velocityField    = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=mesh.dim )
pressureField    = uw.mesh.MeshVariable( mesh=mesh.subMesh, nodeDofCount=1 )

velocityField.data[:]   = [0.,0.]
pressureField.data[:]   = 0.

# Define strain rate tensor

strainRateFn = fn.tensor.symmetric( velocityField.fn_gradient )
strainRate_2ndInvariantFn = fn.tensor.second_invariant(strainRateFn)



### Boundary conditions

Pure shear with moving  side walls — all boundaries are zero traction with outflow top and bottom
to accommodate changing volume

In [4]:
iWalls = mesh.specialSets["MinI_VertexSet"] + mesh.specialSets["MaxI_VertexSet"]
jWalls = mesh.specialSets["MinJ_VertexSet"] + mesh.specialSets["MaxJ_VertexSet"]

leftI  = mesh.specialSets["MinI_VertexSet"]
rightI = mesh.specialSets["MaxI_VertexSet"]
bothIs = leftI + rightI

baseWall   = mesh.specialSets["MinJ_VertexSet"]
topWall    = mesh.specialSets["MaxJ_VertexSet"]

allWalls = iWalls + jWalls

velocityBCs = uw.conditions.DirichletCondition( variable        = velocityField, 
                                                indexSetsPerDof = (iWalls+topWall, jWalls) )

# for index in topWall:
#     velocityField.data[index,0] = 1.0

velocityField.data[topWall.data,0] = np.sin(np.pi * mesh.data[topWall.data,0])
# Already set to zero everywhere else 

In [5]:
stokesPIC = uw.systems.Stokes( velocityField  = velocityField, 
                               pressureField  = pressureField,
                               conditions     = [velocityBCs,],
                               fn_viscosity   = 1.0, 
                               fn_bodyforce   = (0.0,0.0) )

# And a suitable solver package is already attached to it

solver = uw.systems.Solver( stokesPIC )

solver.solve()

In [6]:
import unsupported.triangulation.meshswarm2D 
reload(unsupported.triangulation.meshswarm2D)

mswarm = unsupported.triangulation.meshswarm2D.meshSwarm2D(mesh, velocityField)
mswarmPoints = mesh.data.copy()
mswarm.lagrSwarm.add_particles_with_coordinates(mswarmPoints)

mswarm.update_triangulation()

/Users/lmoresi/+Codes/underworld2-gitology/unsupported/triangulation/__init__.py:9: UserWarning: 

The triangulation module is not supported.
Questions should be addressed to louis.moresi@unimelb.edu.au 


  """


In [7]:
trackVar = unsupported.triangulation.meshswarm2D.meshSwarmVariable2D(mswarm, dataType='float', count=1)
trackVar.lagrangianData.data[:,0] = (1.0-trackVar.lagrangianData.swarm.particleCoordinates.data[:,1])


In [8]:
## Create tracking variables

# import numpy.ma as ma
# If we want to add all the non-boundary nodes to the swarm:
# masked = ma.array(trackVariable.data)
# masked.mask = ma.make_mask_none(masked.shape)
# masked.mask[mesh.specialSets["AllWalls_VertexSet"].data] = True
# non_bound_pts = masked.compressed()


In [9]:
# import matplotlib.pyplot as plt


# figure = plt.figure(figsize=(7,6))
# ax = figure.add_subplot(111)

# s = mswarm
# points = mswarm.lagrSwarm.particleCoordinates.data
# m = ax.tripcolor(points[:,0], points[:,1], trackVar.lagrangianData.data.reshape(-1), alpha=0.6)

# ax.triplot(points[:,0], points[:,1], color="Black", 
#            alpha=0.4, zorder=100)

# plt.colorbar(mappable=m)


glfig = glucifer.Figure(quality=3)

glfig.append( glucifer.objects.Mesh(mesh=mesh, opacity=0.5))
glfig.append( glucifer.objects.Points(swarm=mswarm.lagrSwarm, 
                                      fn_colour=trackVar.lagrangianData,
                                      pointSize=5.0))


In [10]:
#time = 0.0
tri = mswarm.triangulation

for timestep in range(0,1000):
    dt = mswarm.get_max_dt()
    mswarm.integrate(dt)
            
    if timestep % 100 == 0:
        glfig.save_image(filename="SwarmImage"+str(timestep)+".png")
        
    pts = mswarm.prune_swarm_min_length(0.005)
    with mswarm.deform_swarm():
        mswarm.lagrSwarm.particleCoordinates.data[pts,0] = -1.0e99
    print "Deleted ", len(pts), "points -> ", mswarm.triangulation.npoints
       
    pts = mswarm.refine_swarm_max_length(0.025)
    newVals = trackVar.evaluate(pts)
    newpts = mswarm.lagrSwarm.add_particles_with_coordinates(pts)
    mypts  = np.where(newpts != -1)[0] 
    trackVar.lagrangianData.data[newpts[mypts],0] = newVals[mypts]

    print "Added ", len(pts), "points -> ", mswarm.triangulation.npoints
    mswarm.update_triangulation()
      
    pts = mswarm.refine_swarm_too_narrow(5.0)
    newVals = trackVar.evaluate(pts)
    newpts = mswarm.lagrSwarm.add_particles_with_coordinates(pts)
    mypts  = np.where(newpts != -1)[0] 
    trackVar.lagrangianData.data[newpts[mypts],0] = newVals[mypts]

    print "Added ", len(pts), "points -> ", mswarm.triangulation.npoints
    mswarm.update_triangulation()

    # "Boundary" conditions
    
    top_pts = np.where(mswarm.lagrSwarm.particleCoordinates.data[:,1] == 1.0)
    trackVar.lagrangianData.data[top_pts] = 0.0
    
    base_pts = np.where(mswarm.lagrSwarm.particleCoordinates.data[:,1] == 0.0)
    trackVar.lagrangianData.data[base_pts] = 1.0
   

AttributeError: 'module' object has no attribute 'lucColourMap_SetProperties'

In [ ]:
print timestep

In [ ]:
mswarm.update_triangulation()

figure = plt.figure(figsize=(7,6))
ax = figure.add_subplot(111)

s = mswarm
points = mswarm.lagrSwarm.particleCoordinates.data
m = ax.tripcolor(points[:,0], points[:,1], trackVar.lagrangianData.data.reshape(-1), alpha=0.6)

ax.triplot(points[:,0], points[:,1], color="Black", 
           alpha=0.4, zorder=100)

plt.colorbar(mappable=m)

In [ ]:
trackVar.lagrangianData.data.min()

In [ ]:
# figure.savefig("testmesh.png", dpi=600)








In [ ]:
Ffn = fn.math.sin( np.pi * fn.coord()[0]) + fn.math.sin( np.pi * fn.coord()[1])
Fxfn  = np.pi * fn.math.cos( np.pi * fn.coord()[0]) * fn.math.sin( np.pi * fn.coord()[1])
Fyfn  = np.pi * fn.math.sin( np.pi * fn.coord()[0]) * fn.math.cos( np.pi * fn.coord()[1])
Fxxfn = -np.pi**2 * fn.math.sin( np.pi * fn.coord()[0]) * fn.math.sin( np.pi * fn.coord()[1])
Fyyfn = -np.pi**2 * fn.math.sin( np.pi * fn.coord()[0]) * fn.math.sin( np.pi * fn.coord()[1])

In [ ]:
F = np.sin(np.pi * mswarm.triangulation.points[:,0]) * \
    np.sin(np.pi * mswarm.triangulation.points[:,1])
    
FxA   = np.pi * np.cos(np.pi * mswarm.triangulation.points[:,0]) * \
                np.sin(np.pi * mswarm.triangulation.points[:,1])

Fx2A  = -np.pi**2 * np.sin(np.pi * mswarm.triangulation.points[:,0]) * \
                    np.sin(np.pi * mswarm.triangulation.points[:,1])


FyA   = -np.pi * np.sin(np.pi * mswarm.triangulation.points[:,0]) * \
                 np.cos(np.pi * mswarm.triangulation.points[:,1])

Fy2A  = -np.pi**2 * np.sin(np.pi * mswarm.triangulation.points[:,0]) * \
                   np.sin(np.pi * mswarm.triangulation.points[:,1])

    
    

In [ ]:
DX, DY, DXX, DXY, DYY   = mswarm.triangulation_compute_gradient(F, second=True)
DXX2, DYX2 = mswarm.triangulation_compute_gradient(DX)
DXY2, DYY2 = mswarm.triangulation_compute_gradient(DY)

In [ ]:
FxA.max(), DX.max(), (DXX2-DXX).max()

In [ ]:

figure = plt.figure(figsize=(10,10))
ax = figure.add_subplot(111)

s = mswarm

points = mswarm.interpolator.points

m = ax.tripcolor(points[:,0], points[:,1], DY-FyA, alpha=0.6)

ax.triplot(points[:,0], points[:,1], color="Black", 
           alpha=0.4, zorder=100)

plt.colorbar(mappable=m)

# figure.savefig("meshGrad.png", dpi=300)

In [ ]:
del2fn = gradXVar.fn_gradient[0] + gradYVar.fn_gradient[1]




In [ ]:
gptVariable.data[:,0] = trackVar.evaluate(gaussSwarm.particleCoordinates.data)

fig1 = glucifer.Figure( figsize=(600,600) )
fig1.append( glucifer.objects.Mesh(mesh=imesh, opacity=0.25))
fig1.append( glucifer.objects.VectorArrows(mesh=mesh, fn=velocityField, scaling=0.1 ))
fig1.append( glucifer.objects.Surface(mesh=imesh, fn=Fxxfn ))

# fig1.append( glucifer.objects.Points(swarm=gaussSwarm, fn_colour=gptVariable,
#                                      colours="Blue Red", pointSize=0.0))

fig1.show()